# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [25]:
from azureml.core import Workspace, Experiment
from azureml.core import Model 
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from pprint import pprint 
import joblib
import os

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [26]:
#ANCLA:una vez que ya fue leida la data enviar un mensaje de realizado
#ANCLA: enviar una salida de datos del dataframe para que pueda visualizarse
#getting data: the dataset was uploaded to Azure Environment but can be accesible through an online link for example
from azureml.data.dataset_factory import TabularDatasetFactory

rawdata_homic2018 = "https://raw.githubusercontent.com/hftamayo/azuremlproj03/main/cad2018.csv"
dshomic2018 = TabularDatasetFactory.from_delimited_files(path=rawdata_homic2018, separator=',')
pdhomic2018 = dshomic2018.to_pandas_dataframe().dropna()

In [27]:
#ANCLA: hacer un check en caso que el experimento ya exista, enviar un mensaje que fue creado o que ya existe segun sea el caso
#setting up workspace and dataset
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'expp03ht'

experiment=Experiment(ws, experiment_name)

In [28]:
# setting up IaaS where the experiment will run on it
ccname = "ccp03aml"
try:
    ccluster = ComputeTarget(ws, ccname)
    print(f"{ccname} exists or is in use!, choose a different name")
except:
    ccconfig = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5)
    ccluster = ComputeTarget.create(ws, ccname, ccconfig)
ccluster.wait_for_completion(show_output=True)

CreatingAmlCompute is getting created. Consider calling wait_for_completion() first


Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [29]:
#ANCLA: enviar un mensaje que las configs fueron seteadas
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
        task='classification',
        compute_target=ccluster,
        training_data=dshomic2018,
        label_column_name='sexo',
        n_cross_validations=5,
        **automl_settings
)

In [30]:
# TODO: Submit your experiment
exprun = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [31]:
RunDetails(exprun).show()
exprun.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of 

        60   StandardScalerWrapper XGBoostClassifier        0:00:49       0.8790    0.8816
        61   MaxAbsScaler LightGBM                          0:00:50       0.8790    0.8816
        62   StandardScalerWrapper XGBoostClassifier        0:00:47       0.8787    0.8816
        63   SparseNormalizer LightGBM                      0:00:49       0.8790    0.8816
        64   StandardScalerWrapper LightGBM                 0:01:01       0.8790    0.8816
        65   SparseNormalizer XGBoostClassifier             0:00:46       0.8790    0.8816
        66   SparseNormalizer XGBoostClassifier             0:00:51       0.8790    0.8816
        67   MaxAbsScaler LightGBM                          0:00:55       0.8790    0.8816
        68   SparseNormalizer LightGBM                      0:00:47       0.8790    0.8816
        69   StandardScalerWrapper LightGBM                 0:00:36       0.8790    0.8816
        70   StandardScalerWrapper XGBoostClassifier        0:00:31       0.8790    0.8816

{'runId': 'AutoML_4cd1cc7a-8222-45fe-81c2-518b7e62c720',
 'target': 'ccp03aml',
 'status': 'Completed',
 'startTimeUtc': '2021-01-23T18:50:27.824415Z',
 'endTimeUtc': '2021-01-23T19:21:45.479675Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'ccp03aml',
  'AMLSettingsJsonString': '{"path":null,"name":"expp03ht","subscription_id":"aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee","resource_group":"aml-quickstarts-135796","workspace_name":"quick-starts-ws-135796","region":"southcentralus","compute_target":"ccp03aml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featurization":

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [32]:
#import azureml.train.automl
# obtaining best run and fitted model
brmodel, fmodel = exprun.get_output()

# Print the best run
print(brmodel)

# Get all metrics of the best run model
brmetrics = brmodel.get_metrics()

# Print all metrics of the best run model
for metric_name in brmetrics:
    metric = brmetrics[metric_name]
    print(metric_name, metric)
    

Package:azureml-automl-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-core, training version:1.20.0, current version:1.19.0
Package:azureml-dataprep, training version:2.7.2, current version:2.6.1
Package:azureml-dataprep-native, training version:27.0.0, current version:26.0.0
Package:azureml-dataprep-rslex, training version:1.5.0, current version:1.4.0
Package:azureml-dataset-runtime, training version:1.20.0, current version:1.19.0.post1
Package:azureml-defaults, training version:1.20.0, current version:1.19.0
Package:azureml-interpret, training version:1.20.0, current version:1.19.0
Package:azureml-pipeline-core, training version:1.20.0, current version:1.19.0
Package:azureml-telemetry, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-client, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-runtime, training version:1.20.0, current version:1.19.0


Run(Experiment: expp03ht,
Id: AutoML_4cd1cc7a-8222-45fe-81c2-518b7e62c720_71,
Type: azureml.scriptrun,
Status: Completed)
precision_score_weighted 0.8838844330086608
weighted_accuracy 0.9813178209925848
recall_score_weighted 0.882545568123508
log_loss 0.35715977668467025
average_precision_score_micro 0.8885182477326646
f1_score_micro 0.882545568123508
AUC_macro 0.6245011380397539
average_precision_score_macro 0.5703630529572051
accuracy 0.882545568123508
f1_score_weighted 0.8318511379493969
f1_score_macro 0.502185249674696
precision_score_macro 0.887964234405165
AUC_weighted 0.6245011380397539
norm_macro_recall 0.0345564744533132
AUC_micro 0.9053366833475426
recall_score_micro 0.882545568123508
balanced_accuracy 0.5172782372266566
recall_score_macro 0.5172782372266566
precision_score_micro 0.882545568123508
average_precision_score_weighted 0.831132124016438
matthews_correlation 0.156669990924085
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_4cd1cc7a-8222-45fe-81c2-518b7e62c

In [33]:
import joblib
joblib.dump(value=fmodel, filename='model.pkl')

['model.pkl']

In [35]:
#TODO: Save the best model
autobmlp3ht = brmodel.register_model(model_path='outputs/model.pkl', model_name='p3automl_ht',
                        tags={'Method of execution':'Auto ML'},
                        properties={'Accuracy': brmetrics['accuracy']})

print(autobmlp3ht)

Model(workspace=Workspace.create(name='quick-starts-ws-135796', subscription_id='aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee', resource_group='aml-quickstarts-135796'), name=p3automl_ht, id=p3automl_ht:3, version=3, tags={'Method of execution': 'Auto ML'}, properties={'Accuracy': '0.882545568123508'})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [36]:
# Download scoring file 
brmodel.download_file('outputs/scoring_file_v_1_0_0.py', 'scoring.py')

# Download environment file
brmodel.download_file('outputs/conda_env_v_1_0_0.yml', 'enviro.yml')

In [37]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

infcfg = InferenceConfig(entry_script='scoring.py',
                                    environment=Environment.from_conda_specification(name='myenv',file_path='enviro.yml'))

# deploying the model via WebService
from azureml.core.webservice import AciWebservice

depcfg = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
webservice = Model.deploy(ws, "wsp03ht01", [autobmlp3ht], infcfg, depcfg)
webservice.wait_for_deployment(show_output = True)
print(webservice.state)

print(webservice.scoring_uri)

print(webservice.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://b1be3db3-8fa5-43ca-a6fe-a6b74b0e3e45.southcentralus.azurecontainer.io/score
http://b1be3db3-8fa5-43ca-a6fe-a6b74b0e3e45.southcentralus.azurecontainer.io/swagger.json


In [58]:
import json

#Importing the data set for testing 
tsthomic2018 = pdhomic2018.sample(10) 
lblhomic2018 = tsthomic2018.pop('sexo')

tstdatahomic2018 = json.dumps({'data': tsthomic2018.to_dict(orient='records')})

print(tstdatahomic2018)

{"data": [{"id": 3017, "regfalle": 18870, "edad": 38, "deptoocuhe": 9, "tipoarma": 1, "pracaut": 1}, {"id": 791, "regfalle": 19210, "edad": 74, "deptoocuhe": 6, "tipoarma": 1, "pracaut": 0}, {"id": 2311, "regfalle": 23051, "edad": 30, "deptoocuhe": 11, "tipoarma": 9, "pracaut": 1}, {"id": 1280, "regfalle": 21746, "edad": 20, "deptoocuhe": 2, "tipoarma": 9, "pracaut": 1}, {"id": 1042, "regfalle": 21711, "edad": 39, "deptoocuhe": 4, "tipoarma": 1, "pracaut": 0}, {"id": 2503, "regfalle": 24465, "edad": 44, "deptoocuhe": 8, "tipoarma": 1, "pracaut": 1}, {"id": 3001, "regfalle": 22176, "edad": 25, "deptoocuhe": 8, "tipoarma": 1, "pracaut": 1}, {"id": 838, "regfalle": 20172, "edad": 25, "deptoocuhe": 6, "tipoarma": 1, "pracaut": 0}, {"id": 302, "regfalle": 21003, "edad": 36, "deptoocuhe": 11, "tipoarma": 13, "pracaut": 1}, {"id": 481, "regfalle": 20327, "edad": 18, "deptoocuhe": 6, "tipoarma": 2, "pracaut": 1}]}


TODO: In the cell below, send a request to the web service you deployed to test it.

In [59]:
import requests

headers = {'Content-type': 'application/json'}

response = requests.post(webservice.scoring_uri, tstdatahomic2018, headers=headers)

In [60]:
# Print results from the inference
print(response.text)

"{\"result\": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}"


In [61]:
# Print original labels
print(lblhomic2018)

3016    0
790     1
2310    1
1279    0
1041    0
2502    0
3000    0
837     0
301     0
480     1
Name: sexo, dtype: int64


TODO: In the cell below, print the logs of the web service and delete the service

In [62]:
print(webservice.get_logs())

2021-01-23T19:27:32,500497108+00:00 - iot-server/run 
2021-01-23T19:27:32,500722307+00:00 - gunicorn/run 
2021-01-23T19:27:32,502142501+00:00 - nginx/run 
2021-01-23T19:27:32,502862599+00:00 - rsyslog/run 
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [ ]:
webservice.delete()